# **Importing Dependencies**

In [0]:
import pickle
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, TimeDistributed, Dense, Dropout, BatchNormalization, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from tensorflow.python.client import device_lib
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

# **Allowing for Parallelized Model Training**
By default, TensorFlow allocates all available GPU memory to the current training process. By enabling memory growth, however, we can train multiple models in parallel.

In [0]:
gpus = tf.config.experimental.list_physical_devices("GPU")

if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

# **Loading Data & Labels**
Here, we load the features previously extracted by the CNN and their corresponding labels.

In [0]:
data_array = np.load("rcnn-data-array.npy")
label_array = np.load("rcnn-label-array.npy")

# Add a dummy dimension for label_array to be fed to our R-CNN
label_array = np.expand_dims(label_array, axis=2)

In [0]:
# We check the percentage of hemorrhages present in the data: It is ~19%.
np.average(label_array)

0.1935311552247036

## **Building the Model**

In [0]:
with tf.device("/GPU:1"):
    model = Sequential()

    model.add(Bidirectional(LSTM(100, return_sequences=True), input_shape=(slices_per_patient, 8192)))
    model.add(TimeDistributed(Dropout(0.5)))
    model.add(TimeDistributed(Dense(100, use_bias=False)))
    model.add(TimeDistributed(BatchNormalization()))
    model.add(TimeDistributed(Activation("relu")))
    model.add(TimeDistributed(Dropout(0.5)))
    model.add(TimeDistributed(Dense(100, use_bias=False)))
    model.add(TimeDistributed(BatchNormalization()))
    model.add(TimeDistributed(Activation("relu")))
    model.add(TimeDistributed(Dense(1, activation="sigmoid")))
    
    model.compile(loss="binary_crossentropy", optimizer=Adam(lr=0.0001), metrics=model_metrics)

## **Fitting the Model**

In [0]:
# Training setup
model_metrics = ["accuracy",
                 tensorflow.keras.metrics.Precision(),
                 tensorflow.keras.metrics.Recall(),
                 tensorflow.keras.metrics.AUC()]

slices_per_patient = 24
batch_size = 100
epochs = 100
model_name = "bi-rcnn-model"

checkpoint = ModelCheckpoint("RCNN_models/bi-rcnn-model-{epoch:03d}-{val_recall:.2f}.hdf5", 
                             "val_accuracy",
                             save_best_only=False, verbose=2)

stopping = EarlyStopping("val_accuracy", patience = 10, verbose = 2, mode = "max")

callbacks = [stopping, checkpoint]

In [0]:
# Model fitting
history = model.fit(data_array, label_array,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_split=0.2,
                    callbacks=callbacks,
                    verbose=2)

# **Plotting Development of Loss, AUC, Recall**

In [0]:
with open("RNN_histories/bi-rcnn-{}.pkl".format(model_name), "wb") as f:
    pickle.dump(history.history, f)

In [0]:
# Takes in one of "loss", "auc", "recall"
# Plots the development of the corresponding metric
def plot_metric(metric):
  plt.figure(figsize=(10, 6))
  plt.plot(history.history[metric], label=metric)
  plt.plot(history.history["val_" + metric], label="val_" + metric)
  plt.xlabel("epoch")
  plt.ylabel(metric)
  plt.title(model_name + metric)
  plt.legend()
  plt.tight_layout()
  plt.savefig("RNN_plots/bi-rcnn-{}_".format(model_name) + metric + ".png", dpi=300)
  plt.show()

  pass

In [0]:
# Plot of train and test loss
plot_metric("loss")

In [0]:
# Plot of train and test recall
plot_metric("recall")

In [0]:
# Plot of train and test AUC
plot_metric("auc")